### Import files

In [1]:
import numpy as np
import scipy as sp
import pandas as pd

data_loc = "../../data/intact/"

In [5]:
spA = "rat"
spB = "fly"

dfA = pd.read_csv(data_loc + spA + ".s.tsv", sep = "\t", header = None)
dfB = pd.read_csv(data_loc + spB + ".s.tsv", sep = "\t", header = None)

nodesA = set(dfA[0]).union(dfA[1])
nodesB = set(dfB[0]).union(dfB[1])

nA = {k: i for i, k in enumerate(nodesA)}
nB = {k: i for i, k in enumerate(nodesB)}

udfA = dfA.copy()
udfB = dfB.copy()

udfA.loc[:, 0] = udfA.loc[:, 0].apply(lambda x : nA[x])
udfA.loc[:, 1] = udfA.loc[:, 1].apply(lambda x : nA[x])

udfB.loc[:, 0] = udfB.loc[:, 0].apply(lambda x : nB[x])
udfB.loc[:, 1] = udfB.loc[:, 1].apply(lambda x : nB[x])

In [8]:
from scipy.sparse import coo_array

A = coo_array(([0,1,4,5], ([0,3,1,0],[0,3,1,2])), shape=(4,4)) 
A

ImportError: cannot import name 'coo_array' from 'scipy.sparse' (/cluster/tufts/cowenlab/.envs/denoise/lib/python3.7/site-packages/scipy/sparse/__init__.py)